In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Correlation Power Analysis on AES

## AES
![](https://www.researchgate.net/profile/Sameera-Lakmal/publication/315825592/figure/fig5/AS:480906528989188@1491668459167/AES-Encryption-steps-rounds.png)

"SubBytes" (also called SBOX-Lookup) is perfect point for correlation power analysis.

```c
uint8_t sbox(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t *buf)
{
    trigger_high();

    uint8_t output[16];
    for (uint8_t i = 0; i < 16; i++)
    {
        output[i] = aes_sbox[aes_key[i] ^ buf[i]];
    }

    simpleserial_put(0x01, 16, output);
    trigger_low();
    return 0;
}
```

where

```c
static const uint8_t aes_sbox[] = {
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
};
```

In [ ]:
import numpy as np

SBOX = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def sbox(data):
    return SBOX[data]

sbox_vec = np.vectorize(sbox)

## Difficult attack? - No, just repeat the previous attack!

In [ ]:
import securec
import securec.util
scope, target = securec.util.init()
scope.default_setup()

In [ ]:
securec.util.compile_and_flash('./example_4.c')

In [ ]:
scope.default_setup()
securec.util.reset_target()
def capture(attempt, samples=500):
    scope.adc.samples = samples
    scope.arm()
    if isinstance(attempt, int):
        attempt = bytes([attempt])
    elif isinstance(attempt, list):
        attempt = bytes(attempt)
    elif isinstance(attempt, str):
        attempt = attempt.encode()
    target.simpleserial_write(0x01, attempt + (8 - len(attempt)) * b'\x00')
    trace = securec.util.capture()
    return trace, not bool(target.simpleserial_read(0x01)[0])

In [ ]:
import math
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label, LinearColorMapper, Span
from bokeh.palettes import Category10_10

output_notebook()

In [ ]:
import numpy as np

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [ ]:
HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

In [ ]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [ ]:
import random
import tqdm
import tqdm.notebook

def capture_random(trace_samples=500, trace_nums=100):
    traces = []
    attempts = []
    for _ in tqdm.notebook.tqdm(range(trace_nums)):
        attempt = bytes([random.randint(0, 255) for _ in range(10)])
        traces.append(capture(attempt, samples=trace_samples)[0])
        attempts.append(attempt)
    traces = np.array(traces)
    attempts = np.array([list(a) for a in attempts])
    return traces, attempts

In [ ]:
def plot_pearson(
    traces, 
    attempts, 
    intermediates=lambda attempts: hw_vec([a[0] for a in attempts])
):
    pearsons = abs(pearson_pointwise(traces, intermediates(attempts)))
    p = figure(height=300, sizing_mode='stretch_width', y_range=(0, 1))
    p.title = "PCC(hw(attempts), traces[:, i])"
    p.add_tools(CrosshairTool())
    p.line(range(0, len(pearsons)), pearsons)
    show(p)

In [ ]:
traces, attempts = capture_random()

In [ ]:
keyguess = 0x01 # <- Try different guesses
plot_pearson(traces, attempts, intermediates=lambda attempts: [hw(sbox(attempt[0] ^ keyguess)) for attempt in attempts])

### Attack algorithm

In [ ]:
def attack_aes_sbox(
    traces, 
    attempts,
    password_index=0,
):
    max_pearsons = []
    for c in range(256):
        max_pearsons.append((max(abs(pearson_pointwise(
            traces,
            [hw(sbox(attempt[password_index] ^ c)) for attempt in attempts],
        ))), c))
    return sorted(max_pearsons, reverse=True)

In [ ]:
traces, attempts = capture_random()

In [ ]:
attack_aes_sbox(traces, attempts, password_index=0)

## Summary

👍  It is easy to break unprotected AES SBOX-lookup with CPA.

In [ ]:
securec.util.exit()